In [3]:
# Import Python packages
import praw
from datetime import datetime
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI
import sys


In [4]:
# Load required variables from .env file.
load_dotenv(dotenv_path=Path("C:\\Python\\azure-openai-lab\\.venv\\.env")) #Error sometimes due to \ or \\. Try one or the other. "C:\\Python\\azure-openai-lab\\.venv\\.env"

# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
azure_oai_key = os.environ['AZURE_OPENAI_KEY_P34']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT_P34']

reddit_api_id = os.environ['REDDIT_API_ID']
reddit_api_secret = os.environ['REDDIT_API_SECRET']

In [5]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
        azure_endpoint = azure_oai_endpoint, 
        api_key=azure_oai_key,  
        api_version="2024-10-21" #"2024-02-01"
        )

In [64]:


# 1) Authenticate
reddit = praw.Reddit(
    client_id=reddit_api_id,
    client_secret=reddit_api_secret,
    user_agent="CommentRetriever/0.1 by u/No_Way629"
)

# 2) Fetch the submission
submission = reddit.submission(url="https://www.reddit.com/r/MicrosoftFabric/comments/1k0ty76/lakehouse_sql_endpoint/")

# 4) Start building the output string with the post metadata and body
lines = []
post_time = datetime.utcfromtimestamp(submission.created_utc).isoformat()
lines.append(f"Post: {submission.title} (u/{submission.author})")
lines.append(f"Time: {post_time}")
lines.append(f"URL:  {submission.url}\n")
lines.append(submission.selftext or "[no body text]")
lines.append("\n---\nComments:")

# 5) Load all comments
submission.comments.replace_more(limit=None)

# 6) Flatten and append each comment to output
for comment in submission.comments.list():
    c_time = datetime.utcfromtimestamp(comment.created_utc).isoformat()
    author = comment.author or "[deleted]"
    body   = comment.body.replace('\n', ' ')
    lines.append(f"[{c_time}] u/{author}: {body}")

output = "\n".join(lines)

In [62]:
# Create advanced System prompt
systemcontent = \
"""
### INSTRUCTIONS
Persona: Act as social media expert Dr. Rachna Jain who specializes in strategies for using psychological principles to enhance social media content.
Action: Create a social media post telling a story in the active voice that provide practical business value by Focusing on a single and clear Goal. The structure should be authentic and in the third person. The Text should be clear and professional (no emojis)
Target Audience: The recipients of the post are business leaders, managers and professionals who are curious about optimizing their data landscape.

### EXAMPLE

Hidden Costs in Microsoft Fabric

Did you know that mismatched home and capacity regions in Microsoft Fabric can lead to unexpected data transfer costs and performance issues? If your tenant's home region differs from your capacity region, you might be facing:​
Microsoft Fabric Community

1. Increased data transfer expenses: Data moving between regions can incur additional charges.

2. Performance lags: Certain metadata and features like screenshots and data alerts still process in the home region, potentially causing delays.

3. Compliance challenges: Data residency requirements might be compromised, leading to governance concerns.

Solution:

1. Review your tenant's home region: Ensure it aligns with your capacity region to avoid unnecessary costs.
"""

In [65]:
# Zero-Shot learning. Model has a token limit of 4096.

# Send request to Azure OpenAI model
response = client.chat.completions.create(
    model="gpt-4.5-preview",       # or whichever model you prefer
    temperature=0.4,
    messages=[
        {
            "role": "system",
            "content": systemcontent
        },
        {
            "role": "user",
            "content": (
                "Create a post:\n"
                + "---\n"
                + output
            )
        }
    ]
)

result = response.choices[0].message.content
print(result + "\n")

Case Sensitivity Challenges in Lakehouse SQL Endpoint

A company recently migrated over 100 SSRS reports to Power BI paginated reports and modernized their legacy ETL processes using Microsoft Fabric. However, they encountered unexpected errors when connecting paginated reports to their new silver lakehouse via the SQL endpoint. The issue: table names were case-sensitive, causing reports to fail because the SQL queries had inconsistent casing.

Why did this happen?

- Lakehouse SQL endpoints follow the Parquet specification, making table names case-sensitive.
- Existing SQL queries in reports often had inconsistent casing, leading to "table not found" errors.

Practical solutions:

1. Create a Fabric Data Warehouse with case-insensitive collation and build views referencing the lakehouse tables. This allows paginated reports to query tables without casing issues.
2. Temporarily adjust Spark session settings to enforce consistent casing when writing tables to the lakehouse, though this 